In [0]:
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn import model_selection

In [0]:
import dill as pickle

In [0]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [0]:
!cd ..

In [0]:
!ls

drive  sample_data


In [0]:
from google.colab import files
files.upload()

MessageError: ignored

In [0]:
df=pd.read_csv('AllProductReviews.csv')

In [0]:
df.head()

,ReviewTitle,ReviewBody,ReviewStar,Product
0,Honest review of an edm music lover\n,No doubt it has a great bass and to a great ex...,3,boAt Rockerz 255
1,Unreliable earphones with high cost\n,"This earphones are unreliable, i bought it be...",1,boAt Rockerz 255
2,Really good and durable.\n,"i bought itfor 999,I purchased it second time,...",4,boAt Rockerz 255
3,stopped working in just 14 days\n,Its sound quality is adorable. overall it was ...,1,boAt Rockerz 255
4,Just Awesome Wireless Headphone under 1000...😉\n,Its Awesome... Good sound quality & 8-9 hrs ba...,5,boAt Rockerz 255


In [0]:
df.shape

(14337, 4)

In [0]:
df = df[df.ReviewStar != 3]
df.shape

(12834, 4)

In [0]:

def partition(x):
    if x <= 3:
        return 0
    return 1

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = df['ReviewStar']
positiveNegative = actualScore.map(partition) 
df['ReviewStar'] = positiveNegative



In [0]:
df.shape

(12834, 4)

In [0]:
final=df.drop_duplicates(subset={"Product","ReviewBody","ReviewTitle"}, keep='first', inplace=False)
final.shape

(12404, 4)

In [0]:
final_data=final[0:7000]

In [0]:
def textCleaning(df):
    tokens = []
    df=pd.DataFrame(df)
    for i in range(len(df)):
        tokens.append(word_tokenize(df['ReviewBody'].iloc[i]))

    stopwordsList = stopwords.words("english")
    stopwordsList.extend([',','.','-','!'])

    wordsList = []
    for tokenList in tokens:
        words = []
        for word in tokenList:
            if word.lower() not in stopwordsList:
                words.append(word.lower())
        wordsList.append(words)

    wnet = WordNetLemmatizer()
    for i in range(len(wordsList)):
        for j in range(len(wordsList[i])):
            wordsList[i][j] = wnet.lemmatize(wordsList[i][j], pos='v')

    for i in range(len(wordsList)):
        wordsList[i] = ' '.join(wordsList[i])

    return wordsList


In [0]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
wordlist=textCleaning(final_data)

In [0]:
count_vect = CountVectorizer(ngram_range=(1,2)) #in scikit-learn
final_counts = count_vect.fit_transform(final_data['ReviewBody'].values)
final_counts=final_counts.toarray()

In [0]:
y = np.array(final_data['ReviewStar'])

from sklearn.preprocessing import StandardScaler
standardised_data=StandardScaler(with_mean= False ).fit_transform(final_counts)

KeyboardInterrupt: ignored

In [0]:
y = np.array(final_data['ReviewStar'])

In [0]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(final_counts,y,test_size=0.2, random_state=0)

In [0]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(standardised_data,y,test_size=0.2, random_state=0)
myList = list(range(0,50))
neighbors = list(filter(lambda x: x % 2 != 0, myList))
cv_scores = []
# perform 10-fold cross validation
for k in neighbors:
    clf = BernoulliNB(alpha=k, binarize=0.0, fit_prior=True, class_prior=None)
    scores = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

# changing to misclassification error
MSE = [1 - x for x in cv_scores]

# determining best k
optimal_alpha = neighbors[MSE.index(min(MSE))]
print('\nThe optimal alpha is %d.' % optimal_alpha)

# plot misclassification error vs k 
plt.plot(neighbors, MSE)

for xy in zip(neighbors, np.round(MSE,3)):
    plt.annotate('(%s, %s)' % xy, xy=xy, textcoords='data')

plt.xlabel('Number of Neighbors K')
plt.ylabel('Misclassification Error')
plt.show()

print("the misclassification error for each k value is : ", np.round(MSE,3))

KeyboardInterrupt: ignored

In [0]:
clf_optimal = BernoulliNB(alpha=1, binarize=0.0 , fit_prior=False, class_prior=[1,1])

In [0]:
# fitting the model
clf_optimal.fit(X_train, y_train)

BernoulliNB(alpha=1, binarize=0.0, class_prior=[1, 1], fit_prior=False)

In [0]:
pred = clf_optimal.predict(X_test)

# evaluate accuracy
acc = accuracy_score(y_test, pred) * 100
print(acc)

70.5


In [0]:
from sklearn.metrics import confusion_matrix
clf_optimal = BernoulliNB(alpha=optimal_alpha , binarize=0.0 , fit_prior=False, class_prior=[1,1])

# fitting the model
clf_optimal.fit(X_train, y_train)
print(clf_optimal.coef_)
# predict the response
pred = clf_optimal.predict(X_test)

# evaluate accuracy
matrix = confusion_matrix(y_test, pred) 
tn,fp,fn,tp= confusion_matrix(y_test, pred).ravel()
print(tn,fp,fn,tp)
precision=tp/(tp+fp)
recall=tp/(fn+tp)
f1=(2*((precision*recall)/(precision+recall)))
print("recall is:",recall)
print("precision is:",precision)
print("f1 score is:",f1)

print(matrix)


NameError: ignored

In [0]:
review=input()
df = pd.DataFrame()
df['ReviewBody'] = [review]
wordList = textCleaning(df)

KeyboardInterrupt: ignored

In [0]:
vect = count_vect.transform(wordList)
vect=vect.toarray()

NameError: ignored

In [0]:
from sklearn.preprocessing import StandardScaler
standardised_data=StandardScaler(with_mean= False ).fit_transform(vect)

NameError: ignored

In [0]:
pred = clf_optimal.predict(standardised_data)
print(pred)

[1]


In [0]:
if pred[0] == 1:
    print("Positive")
else:
    print("Negative")

Positive


In [0]:
!ls

drive  sample_data


In [0]:
!%cd%

/bin/bash: line 0: fg: no job control


In [0]:
with open('pickle_dump.pk', 'wb') as file:
        pickle.dump(clf_optimal,file)

In [0]:
!ls

drive  pickle_dump.pk  sample_data


In [0]:
from google.colab import files
files.download('pickle_dump.pk')


In [0]:
loaded_model = pickle.load(open('drive/My Drive/pickle_dump_7000.pk', 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.7907142857142857


In [0]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [0]:
y_pred = svclassifier.predict(X_test)

In [0]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[261 122]
 [146 871]]
              precision    recall  f1-score   support

           0       0.64      0.68      0.66       383
           1       0.88      0.86      0.87      1017

    accuracy                           0.81      1400
   macro avg       0.76      0.77      0.76      1400
weighted avg       0.81      0.81      0.81      1400



In [0]:
result = svclassifier.score(X_test, y_test)
print(result)

0.8085714285714286


In [0]:
from google.colab import files
with open('pickle_dump_svm_7k.pk', 'wb') as file:
        pickle.dump(svclassifier,file)

In [0]:
files.download('pickle_dump_svm_7k.pk')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 41268, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [0]:
review=input()
df = pd.DataFrame()
df['ReviewBody'] = [review]
wordList = textCleaning(df)
vect = count_vect.transform(wordList)
vect=vect.toarray()

pred = svclassifier.predict(standardised_data)
print(pred)
if pred[0] == 1:
    print("Positive")
else:
    print("Negative")

In [0]:
!pip install sklearn

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

from sklearn.datasets import *
from sklearn.linear_model import LogisticRegression

tuned_parameters = {'C': np.random.uniform(7008.4868737161105,7010.4868737161105,1000)}
X_train, X_test, y_train, y_test = train_test_split(final_counts, y, train_size=.9)


#Using GridSearchCV
model =  RandomizedSearchCV(LogisticRegression(penalty='l1'), tuned_parameters, scoring = 'accuracy', cv=5)
model.fit(X_train, y_train)

print(model.best_estimator_)
print(model.score(X_test, y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

LogisticRegression(C=7009.136291550662, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
0.8971428571428571


In [0]:
clf = LogisticRegression(random_state = 0,C=7) 
clf.fit(X_train, y_train) 
print(clf.score(X_test, y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8928571428571429


In [0]:
review=input()
df = pd.DataFrame()
df['ReviewBody'] = [review]
wordList = textCleaning(df)
vect = count_vect.transform(wordList)
vect=vect.toarray()

pred = model.predict(vect)
print(pred)
if pred[0] == 1:
    print("Positive")
else:
    print("Negative")

bad product
[0]
Negative


In [0]:
pred = model.predict(X_test)

# evaluate accuracy
matrix = confusion_matrix(y_test, pred) 
tn,fp,fn,tp= confusion_matrix(y_test, pred).ravel()
print(tn,fp,fn,tp)
precision=tp/(tp+fp)
recall=tp/(fn+tp)
f1=(2*((precision*recall)/(precision+recall)))
print("recall is:",recall)
print("precision is:",precision)
print("f1 score is:",f1)

print(matrix)


29 5 4 62
recall is: 0.9393939393939394
precision is: 0.9253731343283582
f1 score is: 0.9323308270676692
[[29  5]
 [ 4 62]]


In [0]:
with open('pickle_dump_LR.pk', 'wb') as file:
        pickle.dump(model,file)
from google.colab import files
files.download('pickle_dump_LR.pk')
